# Import Library

In [ ]:
import numpy as np
from collections import Counter
import math
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')

In [ ]:
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.5

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.2)
Requirement already up-to-date: dill in /usr/local/lib/python3.6/dist-packages (0.3.2)
Requirement already up-to-date: nltk==3.5 in /usr/local/lib/python3.6/dist-packages (3.5)


# Bài 1: 

### Thêm phương pháp Good Turing smoothing vào mô hình ngôn ngữ 1-gram, 2-gram, 3-gram 


In [ ]:
!wget "https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Lecture%202%20Language%20Models/HW2_F17_NLP6320-NLPCorpusTreebank2Parts-CorpusA-Unix.txt"
def readData(fileName):
	data = []
	file = open(fileName, "r")
	for word in file.read().split():
		data.append(word)
	file.close()
	return data

--2021-01-18 15:59:12--  https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Lecture%202%20Language%20Models/HW2_F17_NLP6320-NLPCorpusTreebank2Parts-CorpusA-Unix.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161867 (158K) [text/plain]
Saving to: ‘HW2_F17_NLP6320-NLPCorpusTreebank2Parts-CorpusA-Unix.txt’

HW2_F17_NLP6320-NLP 100%[===================>] 158.07K  --.-KB/s    in 0.01s   

2021-01-18 15:59:12 (12.1 MB/s) - ‘HW2_F17_NLP6320-NLPCorpusTreebank2Parts-CorpusA-Unix.txt’ saved [161867/161867]



In [ ]:
def createBigram(data):
	listOfBigrams = []
	bigramCounts = {}
	unigramCounts = {}
	nbyn = {}
	for i in range(len(data)):
		if i < len(data) - 1:
			listOfBigrams.append((data[i], data[i + 1]))
			if (data[i], data[i+1]) in bigramCounts:
				bigramCounts[(data[i], data[i + 1])] += 1
			else:
				bigramCounts[(data[i], data[i + 1])] = 1
		if data[i] in unigramCounts:
			unigramCounts[data[i]] += 1
		else:
			unigramCounts[data[i]] = 1
	return listOfBigrams, unigramCounts, bigramCounts

In [ ]:

# ------------------------------ Simple Bigram Model --------------------------------
def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):

	listOfProb = {}
	for bigram in listOfBigrams:
		word1 = bigram[0]
		word2 = bigram[1]
		listOfProb[bigram] = (bigramCounts.get(bigram))/(unigramCounts.get(word1))
	return listOfProb

In [ ]:
# ------------------------------- Add One Smoothing ---------------------------------
def addOneSmothing(listOfBigrams, unigramCounts, bigramCounts):
	listOfProb = {}
	cStar = {}
	for bigram in listOfBigrams:
		word1 = bigram[0]
		word2 = bigram[1]
		listOfProb[bigram] = (bigramCounts.get(bigram) + 1)/(unigramCounts.get(word1) + len(unigramCounts))
		cStar[bigram] = (bigramCounts[bigram] + 1) * unigramCounts[word1] / (unigramCounts[word1] + len(unigramCounts))
	return listOfProb, cStar

In [ ]:
def goodTuringDiscounting(listOfBigrams, bigramCounts, totalNumberOfBigrams):
	listOfProb = {}
	bucket = {}
	bucketList = []
	cStar = {}
	pStar = {}
	listOfCounts = {}
	i = 1
	for bigram in bigramCounts.items():
		key = bigram[0]
		value = bigram[1]
		
		if not value in bucket:
			bucket[value] = 1
		else:
			bucket[value] += 1	
	bucketList = sorted(bucket.items() , key=lambda t : t[0])
	zeroOccurenceProb = bucketList[0][1] / totalNumberOfBigrams
	lastItem = bucketList[len(bucketList)-1][0]
	for x in range(1, lastItem):
		if x not in bucket:
			bucket[x] = 0
	bucketList = sorted(bucket.items() , key=lambda t : t[0])
	lenBucketList = len(bucketList)
	for k, v in bucketList:
		if i < lenBucketList-1:
			if v == 0:
				cStar[k] = 0
				pStar[k] = 0
			else:
				cStar[k] = (i+1) * bucketList[i][1] / v
				pStar[k] = cStar[k] / totalNumberOfBigrams
		else:
			cStar[k] = 0
			pStar[k] = 0
		i += 1
	for bigram in listOfBigrams:
		listOfProb[bigram] = pStar.get(bigramCounts[bigram])
		listOfCounts[bigram] = cStar.get(bigramCounts[bigram])
	return listOfProb, zeroOccurenceProb, listOfCounts

In [ ]:
if __name__ == '__main__':
	fileName = 'HW2_F17_NLP6320-NLPCorpusTreebank2Parts-CorpusA-Unix.txt'
	data = readData(fileName)
	listOfBigrams, unigramCounts, bigramCounts = createBigram(data)
	bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)
	bigramAddOne, addOneCstar = addOneSmothing(listOfBigrams, unigramCounts, bigramCounts)
	bigramGoodTuring, zeroOccurenceProb, goodTuringCstar = goodTuringDiscounting(listOfBigrams, bigramCounts, len(listOfBigrams))

	# ------------------------------------- Testing --------------------------------------
	input = "Richard W. Lock , retired vice president and treasurer of"
	inputList = []
	outputProb1 = 1
	outputProb2 = 1
	outputProb3 = 1
	for i in range(len(input.split())-1):
		inputList.append((input.split()[i], input.split()[i+1]))
	print (inputList)
	# ------------------------------ Simple Bigram Model --------------------------------
	for i in range(len(inputList)):
		if inputList[i] in bigramProb:
			outputProb1 *= bigramProb[inputList[i]]
		else:
			outputProb1 *= 0

	print ('Bigram Model: ', outputProb1)

	# ------------------------------- Add One Smoothing ---------------------------------
	for i in range(len(inputList)):
		if inputList[i] in bigramAddOne:
			outputProb2 *= bigramAddOne[inputList[i]]
		else:
			if inputList[i][0] not in unigramCounts:
				unigramCounts[inputList[i][0]] = 1
			prob = (1) / (unigramCounts[inputList[i][0]] + len(unigramCounts))
			addOneCStar = 1 * unigramCounts[inputList[i][0]] / (unigramCounts[inputList[i][0]] + len(unigramCounts))
			outputProb2 *= prob
	print ('Add One: ', outputProb2)

	# ---------------------------- Good Turing Discounting ------------------------------
	for i in range(len(inputList)):
		if inputList[i] in bigramGoodTuring:
			outputProb3 *= bigramGoodTuring[inputList[i]]
		else:
			outputProb3 *= zeroOccurenceProb
	print ('Good Turing: ' , outputProb3)

[('Richard', 'W.'), ('W.', 'Lock'), ('Lock', ','), (',', 'retired'), ('retired', 'vice'), ('vice', 'president'), ('president', 'and'), ('and', 'treasurer'), ('treasurer', 'of')]
Bigram Model:  1.3484074717156563e-10
Add One:  4.602394835477551e-29
Good Turing:  1.906026590289504e-40


# Bài 2:  
### Nhập vào 3 từ đầu tiên của một câu, sinh thêm N từ tiếp theo (N tự đưa vào)

## Download data

In [ ]:
!wget https://github.com/nthanhkhang/Natural-Language-Processing/blob/main/Data/Train_Full.zip
!unzip /content/Train_Full.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train_Full/Van hoa/VH_TN_T_ (11395).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11396).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11398).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11399).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11400).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11401).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11405).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11412).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11413).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11414).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11415).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11417).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11418).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11419).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11421).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11423).txt  
  inflating: Train_Full/Van hoa/VH_TN_T_ (11425).txt  
  inflating: T

In [ ]:
import os

full = []
for dirname, _, filenames in os.walk('/content/Train_Full'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        with open(os.path.join("/content", os.path.join(dirname, filename)), 'r', encoding='UTF-16') as f:
            full.append(f.read())

print(len(full))

42744


In [ ]:
full[13]

'Rụng tóc nhiều ở nữ\n"Cháu đang học lớp 11, kinh nguyệt bình thường, đầu không ngứa, nhưng không hiểu vì sao 5-6 tháng nay tóc của cháu bị rụng rất nhiều. Xin bác sĩ cho biết cháu bị bệnh gì? Nên ăn những chất gì và giữ gìn tóc thế nào để không bị rụng nhiều nữa?".Trả lời: \nNhiều em gái ở độ tuổi cháu cũng bị rụng tóc như vậy - vì vào lứa tuổi dậy thì, cơ thể bị thiếu vitamin B5 (còn có tên là acid pantotenic). Tóc chịu nhiều ảnh hưởng của các yếu tố thần kinh, nội tiết, dinh dưỡng, đặc biệt là vitamin B5. Nhu cầu về vitamin B5 của cơ thể được cung cấp đủ bởi một chế độ ăn bình thường, cân đối. Nhưng đến tuổi dậy thì, rất nhiều bộ phận trong cơ thể phát triển mạnh (tuyến vú, mông, cơ quan sinh dục...) nên tiêu thụ nhiều vitamin B5 hơn, mà thức ăn hằng ngày không đủ cung cấp. Hậu quả là tóc, đặc biệt nhú tóc bị thiếu vitamin B5 nên ngừng mọc và rụng nhiều.Như vậy các em ở tuổi dậy thì nếu tự nhiên thấy rụng nhiều tóc, khiến cho mật độ tóc giảm hẳn đi thì cần nghĩ tới việc thiếu hụt vi

## Preprocess

In [ ]:
import re
from nltk import word_tokenize, sent_tokenize
import string
from tqdm import tqdm

def tokenize(doc):
    tokens = word_tokenize(doc.lower())
    table = str.maketrans('', '', string.punctuation.replace("_", "")) 
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

full_data = ". ".join(full)
full_data = full_data.replace("\n", ". ")
corpus = []
sents = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', full_data)
for sent in tqdm(sents):
    corpus.append(tokenize(sent))

100%|██████████| 738597/738597 [03:56<00:00, 3121.34it/s]


In [ ]:
print(len(corpus))
for c in corpus[:10]:
    print(c)

738597
['trao', 'giải', 'cuộc', 'thi', 'an', 'toàn', 'giao', 'thông', 'cho', 'giới', 'trẻ']
['sáng', 'nay', 'báo', 'sinh', 'viên', 'việt', 'nam', 'và', 'ngân', 'hàng', 'phát', 'triển', 'châu', 'á', 'adb', 'đã', 'trao', 'giải', 'thưởng', 'cho', 'các', 'tác', 'giả', 'đoạt', 'giải', 'cuộc', 'thi', 'viết', 'về', 'an', 'toàn', 'giao', 'thông', 'mang', 'tên', 'hành', 'trình', 'khó', 'quên']
['cuộc', 'thi', 'này', 'thu', 'hút', 'hơn', '5800', 'bài', 'dự', 'thi', 'của', 'học', 'sinh', 'sinh', 'viên', 'các', 'trường', 'trên', 'địa', 'bàn', 'hà', 'nội']
['giải', 'nhất', 'trị', 'giá', '5', 'triệu', 'đồng', 'đã', 'được', 'trao', 'cho', 'em', 'nguyễn', 'khánh', 'dương', 'đại', 'học', 'sư', 'phạm', 'với', 'bài', 'viết', 'hành', 'trình', 'tình', 'nguyện']
['câu', 'chuyện', 'của', 'dương', 'kể', 'lại', 'một', 'buổi', 'chiều', 'trên', 'đường', 'về', 'nhà', 'em', 'đã', 'gặp', 'cảnh', 'ùn', 'tắc', 'giao', 'thông', 'tại', 'nút', 'daewoo', 'kim', 'mã', 'đang', 'khoác', 'trên', 'người', 'chiếc', 'áo', 'xanh


## Train model

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, Laplace, KneserNeyInterpolated, WittenBellInterpolated
import pickle
import os 

n = 3
vi_model = KneserNeyInterpolated(n)

train_data, padded_sents = padded_everygram_pipeline(n, corpus)
vi_model.fit(train_data, padded_sents)
print(len(vi_model.vocab))

model_dir = "/content/drive/My Drive/Colab Notebooks/Ngram_model"
with open(os.path.join(model_dir, 'kneserney_1st_ngram_model.pkl'), 'wb') as fout:
    pickle.dump(vi_model, fout)

104813


In [ ]:
# Test to get logscore
print(vi_model.logscore('nam', ['sinh', 'việt']))
print(vi_model.logscore('nam', ['sinh', 'chuỹen']))
print(vi_model.logscore('nam', ['việt']))

-0.008768946173740685
-16.677458140476162
-0.4283570725364177


In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize
def generate_sent(model, num_words, pre_words=[]):
    content = pre_words
    for i in range(num_words):
        token = model.generate(1, text_seed=content[-2:])
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

generate_sent(vi_model, 10, ["đất", "nước"])

'đất nước việt nam các phương tiện như ở vn sử dụng'

# Bài 2: (optional)

### Viết lại thuật toán tính Bleu score, là thuật toán đo độ tương tự giữa 2 câu:

### https://en.wikipedia.org/wiki/BLEU

In [ ]:
def n_gram_generator(sentence,n= 2,n_gram= False):
    sentence = sentence.lower()
    sent_arr = np.array(sentence.split())
    length = len(sent_arr)
    word_list = []
    for i in range(length+1):
        if i < n:
            continue
        word_range = list(range(i-n,i))
        s_list = sent_arr[word_range]
        string = ' '.join(s_list) 
        word_list.append(string)
        if n_gram:
            word_list = list(set(word_list))
    return word_list

In [ ]:
def bleu_score(original, machine_translated):
    mt_length = len(machine_translated.split())
    o_length  = len(original.split())
    if mt_length > o_length:
        BP=1
    else:
        penality=1-(mt_length/o_length)
        BP = np.exp(penality)
    clipped_precision_score = []
    for ngram_level in range(1, 5):  
        original_ngram_list = n_gram_generator(original, ngram_level)
        original_n_gram = Counter(original_ngram_list)      
        machine_ngram_list = n_gram_generator(machine_translated, ngram_level)
        machine_n_gram = Counter(machine_ngram_list)
        num_ngrams_in_translation = sum(machine_n_gram.values()) 
        for j in machine_n_gram:
            if j in original_n_gram:  
                if machine_n_gram[j] > original_n_gram[j]: 
                    machine_n_gram[j] = original_n_gram[j]      
            else:
                machine_n_gram[j] = 0
        clipped_precision_score.append(float(sum(machine_n_gram.values())) / num_ngrams_in_translation)
    weights = [0.25]*4

    s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, clipped_precision_score))
    s = BP * math.exp(math.fsum(s))
    return s

In [ ]:
if __name__ == "__main__":
  raw = "It is a guide to action which ensures that the military alwasy obeys the command of the party"
  test = "It is the guiding principle which guarantees the military forces alwasy being under the command of the party"
  print ("bleu_score:",bleu_score(raw, test))
  print ("sentence_bleu:",sentence_bleu([raw.split()], test.split()))

bleu_score: 0.27098211583470044
sentence_bleu: 0.27098211583470044
